## Agent Communication Architecture

In [ ]:
import asyncio
import json
import uuid
from datetime import datetime
from typing import Dict, List, Any
from dataclasses import dataclass

print("ADK AGENT COMMUNICATION")
print("=" * 30)

@dataclass
class SimpleTask:
    task_id: str
    task_type: str
    payload: Dict[str, Any]
    result: Any = None

print("Core types ready")